In [1]:
import pandas as pd
import numpy as np

## Institutes

In [2]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df = df.drop_duplicates()
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII179331,Paliano,Casa di reclusione,153,1,65,58,66,17,14,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII179299,Padova Nuovo Complesso,Casa di reclusione,438,63,555,316,336,27,30,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181870,Verbania,Casa circondariale,53,0,83,43,44,5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII180348,Roma Rebibbia femminile,Casa circondariale - Germana Stefanini,272,7,370,203,214,24,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


### Remove outliers

In [3]:
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII179331,Paliano,Casa di reclusione,153,1,65,58,66,17,14,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII179299,Padova Nuovo Complesso,Casa di reclusione,438,63,555,316,336,27,30,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181870,Verbania,Casa circondariale,53,0,83,43,44,5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII180348,Roma Rebibbia femminile,Casa circondariale - Germana Stefanini,272,7,370,203,214,24,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


In [4]:
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'])
df = df.sort_values(by='dati aggiornati al')

In [5]:
# 1. Group by 'dati aggiornati al' and sum 'posti regolamentari'
daily_total_posti = df.groupby('dati aggiornati al')['posti regolamentari'].sum()
daily_total_posti

dati aggiornati al
2024-10-05    51196
2024-10-06    51196
2024-10-07    51196
2024-10-09    51195
2024-10-10    51195
              ...  
2025-07-23    51275
2025-07-24    51275
2025-07-28    51275
2025-07-29    50912
2025-07-30    49380
Name: posti regolamentari, Length: 240, dtype: int64

In [6]:
# 2. Calculate the percentage difference from the previous day's total
# pct_change() is perfect for this.
daily_total_posti_diff_pct = daily_total_posti.pct_change().abs()
daily_total_posti_diff_pct.tail(20)

dati aggiornati al
2025-07-08    0.000000
2025-07-09    0.051369
2025-07-10    0.054130
2025-07-11    0.005285
2025-07-12    0.000471
2025-07-13    0.005787
2025-07-14    0.000000
2025-07-15    0.000000
2025-07-16    0.000000
2025-07-17    0.000000
2025-07-18    0.000000
2025-07-19    0.000000
2025-07-20    0.000000
2025-07-21    0.000000
2025-07-22    0.000000
2025-07-23    0.000000
2025-07-24    0.000000
2025-07-28    0.000000
2025-07-29    0.007079
2025-07-30    0.030091
Name: posti regolamentari, dtype: float64

In [7]:
# Define the percentage threshold for outlier
percentage_threshold = 0.30

In [8]:
bad_dates = daily_total_posti_diff_pct[daily_total_posti_diff_pct > percentage_threshold].index.tolist()
bad_dates

[]

In [9]:
# Removing false positives
# This assumes the pattern: [True Outlier, False Positive, True Outlier, False Positive, ...]
dates_to_remove = [date for i, date in enumerate(bad_dates) if i % 2 == 0]
dates_to_remove

[]

In [10]:
df_cleaned = df[~df['dati aggiornati al'].isin(dates_to_remove)].copy()
df_cleaned

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
121,MII180426,Salerno,Casa circondariale - Antonio Caputo,376,0,599,230,209,30,31,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
122,MII179842,Piacenza,Casa circondariale - San Lazzaro,416,9,494,255,215,19,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
123,MII174686,Ferrara,Casa circondariale - Costantino Satta,244,0,382,181,194,17,19,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
124,MII180001,Rimini,Casa circondariale,118,0,155,109,127,14,17,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45423,MII180348,Roma Rebibbia femminile,Casa circondariale - Germana Stefanini,272,21,369,181,214,21,26,...,94.0,87.0,8.0,132.0,171.0,169.0,2025-07-30,2025-06-30,2025-06-30,2025-07-30
45424,MII181870,Verbania,Casa circondariale,53,0,67,42,44,6,13,...,34.0,5.0,1.0,37.0,37.0,35.0,2025-07-30,2025-06-30,2025-06-30,2025-07-30
45425,MII179299,Padova Nuovo Complesso,Casa di reclusione,438,38,611,313,336,28,30,...,72.0,0.0,3.0,385.0,24.0,24.0,2025-07-30,2025-06-30,2025-06-30,2025-07-30
45416,MII179331,Paliano,Casa di reclusione,153,1,63,60,66,12,14,...,46.0,11.0,0.0,57.0,55.0,56.0,2025-07-30,2025-06-30,2025-06-30,2025-07-30


In [11]:
df_cleaned.to_csv('../outputs/clean/institutes.csv', index=False, encoding='UTF-8-sig')
df = pd.read_csv('../outputs/clean/institutes.csv')


## Most recent data

In [12]:
# Get the most recent update
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id istituto')['dati aggiornati al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti disponibili'] = df_most_recent['posti regolamentari'] - df_most_recent['posti non disponibili']
df_most_recent['tasso di affollamento'] = round(((df_most_recent['totale detenuti'] / df_most_recent['posti disponibili'])*100),0)

In [13]:
df_most_recent.value_counts(['dati aggiornati al'])

dati aggiornati al
2025-07-30            186
2025-07-29              4
Name: count, dtype: int64

In [14]:
df_most_recent.columns

Index(['id istituto', 'nome istituto', 'tipo istituto', 'posti regolamentari',
       'posti non disponibili', 'totale detenuti',
       'polizia penitenziaria - effettivi', 'polizia penitenziaria - previsti',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'educatori - effettivi', 'educatori - previsti', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'dati aggiornati al',
       'personale polizia penitenziaria aggiornato al',
       'personale amministrativo aggiornato al',
       'data di aggiornamento spazi detentivi', 'posti disponibili',
       'tasso di affollamento'],
      dtype='object')

In [15]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id istituto'})

merged_df = pd.merge(df_most_recent, df_info, on='id istituto')

# Adding additional columns
merged_df['posti disponibili'] = merged_df['posti regolamentari'] - merged_df['posti non disponibili']

merged_df['tasso di affollamento'] = round((merged_df['totale detenuti'] / merged_df['posti disponibili']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id istituto',
    'nome istituto',
    'tasso di affollamento',
    'indirizzo',
    'tipo istituto',
   'posti regolamentari',
    'posti non disponibili',
    'posti disponibili',
    'totale detenuti',
    'dati aggiornati al',
    'polizia penitenziaria - previsti',
    'polizia penitenziaria - effettivi',
    'personale polizia penitenziaria aggiornato al',
    'amministrativi - effettivi',
    'amministrativi - previsti',
    'personale amministrativo aggiornato al',
    'numero complessivo',
    'numero non disponibili',
    'doccia',
    'bidet',
    'portatori di handicap',
    'servizi igienici con porta',
    'accensione luce autonoma',
    'prese elettriche',
    'data di aggiornamento spazi detentivi',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head(2)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude
0,MII152284,Genova Marassi,125.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,2,533,667,2025-07-30,...,1.0,128.0,175.0,3.0,213.0,100.0,0.0,2025-07-30,44.417913,8.951100
1,MII157783,Alba,112.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,55,2025-07-30,...,86.0,22.0,112.0,1.0,112.0,22.0,22.0,2025-07-30,44.691542,8.025636


In [16]:
df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


df_filtered.head()

/tmp/ipykernel_3818/3401019820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude,scheda istituto
0,MII152284,Genova Marassi,125.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,2,533,667,2025-07-30,...,128.0,175.0,3.0,213.0,100.0,0.0,2025-07-30,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,112.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,55,2025-07-30,...,22.0,112.0,1.0,112.0,22.0,22.0,2025-07-30,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,217.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,219,483,1047,2025-07-30,...,272.0,184.0,0.0,327.0,325.0,325.0,2025-07-30,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,133.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,109,2025-07-30,...,20.0,20.0,0.0,50.0,40.0,40.0,2025-07-30,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,136.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,24,259,352,2025-07-30,...,192.0,42.0,0.0,247.0,228.0,0.0,2025-07-30,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [17]:
# Create a copy first to avoid the warning
df_filtered = df_filtered.copy()

# Calculate metrics using loc for proper assignment
df_filtered.loc[:, 'stanze_disponibili'] = df_filtered['numero complessivo'] - df_filtered['numero non disponibili']

# Create list of columns to process
metrics = {
    'detenuti_stanza': lambda x: (x['totale detenuti'] / x['stanze_disponibili']).round(2),
    'polizia_pers': lambda x:  (x['polizia penitenziaria - effettivi']/ x['totale detenuti']).round(2),
    'doccia_pers': lambda x:  (x['totale detenuti'] / x['doccia']).round(2),
    'bidet_pers': lambda x:  (x['totale detenuti'] / x['bidet']).round(2),
    'servizi_pers': lambda x:  (x['totale detenuti'] / x['servizi igienici con porta']).round(2),
    'luci_pers': lambda x:  (x['totale detenuti'] / x['accensione luce autonoma']).round(2),
    'prese_pers': lambda x: (x['totale detenuti'] / x['prese elettriche']).round(2)
}

# Apply calculations
for col, func in metrics.items():
    df_filtered.loc[:, col] = func(df_filtered).replace([np.inf, -np.inf], np.nan)

In [18]:
df_filtered.sample(4)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,longitude,scheda istituto,stanze_disponibili,detenuti_stanza,polizia_pers,doccia_pers,bidet_pers,servizi_pers,luci_pers,prese_pers
165,MII181440,Trani femminile,134.0,Piazza Plebiscito n.18 - 76125 Trani,Casa di reclusione,32,0,32,43,2025-07-30,...,16.395945,"<a href=""https://www.giustizia.it/giustizia/pa...",19.0,2.26,0.00,NaN,2.26,2.26,2.26,NaN
189,MII182919,Pisa,150.0,Via S. Giovanni Bosco n.43 - 56127 Pisa,Casa circondariale,195,6,189,284,2025-07-30,...,10.414029,"<a href=""https://www.giustizia.it/giustizia/pa...",147.0,1.93,0.64,10.14,2.49,3.34,2.31,1.91
111,MII179306,Palermo Pagliarelli,122.0,Piazza Pietro Cerulli n.1 - 90129 Palermo,Casa circondariale - Antonio Lorusso,1165,68,1097,1334,2025-07-30,...,13.339320,"<a href=""https://www.giustizia.it/giustizia/pa...",709.0,1.88,0.47,4.48,1.85,1.83,1.85,20.21
149,MII181079,Sant'Angelo dei Lombardi,153.0,C. da Selvatico n. SNC - 83054 Sant'Angelo dei...,"Casa di reclusione - Famiglietti, Forgetta, Ba...",124,0,124,190,2025-07-30,...,15.189285,"<a href=""https://www.giustizia.it/giustizia/pa...",86.0,2.21,0.46,2.24,NaN,2.21,2.21,2.21


In [19]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [20]:
grouped_df = df.groupby('dati aggiornati al').sum(numeric_only=True).reset_index()
grouped_df

,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2025-07-23,51275,4472,62416,31024,34035,3245,4086,930,1003,31911.0,2828.0,14368.0,13717.0,434.0,30386.0,24524.0,21243.0
236,2025-07-24,51275,4474,62434,31024,34035,3245,4086,930,1003,31911.0,2830.0,14368.0,13717.0,434.0,30386.0,24524.0,21243.0
237,2025-07-28,51275,4451,62460,31024,34035,3245,4086,930,1003,31910.0,2840.0,14377.0,13708.0,435.0,30385.0,24523.0,21234.0
238,2025-07-29,50912,4402,61824,30793,33791,3217,4057,923,996,31697.0,2816.0,14306.0,13695.0,434.0,30187.0,24516.0,21023.0


In [21]:
grouped_df['posti disponibili'] = (grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']).round(0)
grouped_df['tasso di affollamento'] = (grouped_df['totale detenuti'] / grouped_df['posti disponibili'] * 100).round(4).astype(float)

grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,posti disponibili,tasso di affollamento
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.1883
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.2567
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46755,132.2768
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46756,132.3103
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46758,132.2619


In [22]:
grouped_df = grouped_df[['dati aggiornati al', 'posti regolamentari', 'posti non disponibili', 'posti disponibili', 'totale detenuti', 'tasso di affollamento']]
# grouped_df['posti disponibili'] = grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']
# grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
0,2024-10-05,51196,4384,46812,61880,132.1883
1,2024-10-06,51196,4384,46812,61912,132.2567
2,2024-10-07,51196,4441,46755,61846,132.2768
3,2024-10-09,51195,4439,46756,61863,132.3103
4,2024-10-10,51195,4437,46758,61843,132.2619


In [23]:
grouped_df.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
238,2025-07-29,50912,4402,46510,61824,132.9263
239,2025-07-30,49380,4506,44874,60506,134.8353


In [24]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [25]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')
df.columns

Index(['id istituto', 'nome istituto', 'tasso di affollamento', 'indirizzo',
       'tipo istituto', 'posti regolamentari', 'posti non disponibili',
       'posti disponibili', 'totale detenuti', 'dati aggiornati al',
       'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi',
       'personale polizia penitenziaria aggiornato al',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'personale amministrativo aggiornato al', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'data di aggiornamento spazi detentivi',
       'latitudine', 'longitude', 'scheda istituto', 'stanze_disponibili',
       'detenuti_stanza', 'polizia_pers', 'doccia_pers', 'bidet_pers',
       'servizi_pers', 'luci_pers', 'prese_pers'],
      dtype='object')

In [26]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome istituto', 'totale detenuti', 'tasso di affollamento', 'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi', 'personale polizia penitenziaria aggiornato al', 'dati aggiornati al']]

df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']

df_polizia.head(2)

/tmp/ipykernel_3818/2362681573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante
0,Genova Marassi,667,125.0,336,321,2025-06-30,2025-07-30,15
1,Alba,55,112.0,107,82,2025-06-30,2025-07-30,25


In [27]:
df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)
df_polizia.head(2)


/tmp/ipykernel_3818/989766878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Genova Marassi,667,125.0,336,321,2025-06-30,2025-07-30,15,4.46
1,Alba,55,112.0,107,82,2025-06-30,2025-07-30,25,23.36


In [28]:
df_polizia_clean = df_polizia[df_polizia['totale detenuti'] != 0]
df_polizia_clean.sort_values('polizia penitenziaria - mancante percentuale', ascending=False).head(2)

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
138,Roma Rebibbia,291,104.0,189,132,2025-06-30,2025-07-30,57,30.16
175,Vasto,101,69.0,95,67,2025-06-30,2025-07-30,28,29.47


In [29]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso di affollamento'] > 120) & (df_polizia_clean['polizia penitenziaria - mancante percentuale'] > 20)].reset_index(drop=True)
critical_prisons

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Ancona Montacuto,334,131.0,175,139,2025-06-30,2025-07-30,36,20.57
1,Augusta,598,175.0,224,170,2025-06-30,2025-07-30,54,24.11
2,Rieti,503,174.0,175,129,2025-06-30,2025-07-30,46,26.29
3,Cassino,163,177.0,142,102,2025-06-30,2025-07-30,40,28.17
4,Matera,180,144.0,125,94,2025-06-30,2025-07-30,31,24.80
5,Pescara,390,160.0,166,120,2025-06-30,2025-07-30,46,27.71
6,Roma Regina Coeli,1097,192.0,480,358,2025-06-30,2025-07-30,122,25.42
7,Viterbo,705,174.0,330,253,2025-06-30,2025-07-30,77,23.33


In [30]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [31]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [32]:
df2.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
238,2025-07-29,50912,4402,46510,61824,132.9263
239,2025-07-30,49380,4506,44874,60506,134.8353


In [33]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'dati aggiornati al': 'Date', 'tasso di affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [34]:
# Converting the Date columns to datetime for consistency
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [35]:
df2.tail(2)

,Date,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso_affollamento_reale
238,2025-07-29,50912,4402,46510,61824,132.9263
239,2025-07-30,49380,4506,44874,60506,134.8353


In [36]:
merged_df = pd.merge(df1[['Date', 'tasso_affollamento_ufficiale']], 
                     df2[['Date', 'tasso_affollamento_reale']], 
                     on='Date', 
                     how='outer')
merged_df.sort_values(by='Date', inplace=True)

merged_df.tail(2)

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
308,2025-07-29,NaN,132.9263
309,2025-07-30,NaN,134.8353


In [37]:
filtered_df = merged_df[merged_df['Date'] > '2024-08-01']
filtered_df.tail(2)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
308,2025-07-29,NaN,132.9263
309,2025-07-30,NaN,134.8353


In [38]:
# Apply linear interpolation for missing values
filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)

filtered_df.tail(2)


/tmp/ipykernel_3818/3090938612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
/tmp/ipykernel_3818/3090938612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale,tasso_affollamento_ufficiale (interpolated),tasso_affollamento_reale (interpolated)
308,2025-07-29,NaN,132.9263,120.694,132.9263
309,2025-07-30,NaN,134.8353,120.694,134.8353


In [39]:
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)

In [40]:
df_reale = filtered_df[['Date', 'tasso_affollamento_reale']]
df_reale = df_reale[df_reale['tasso_affollamento_reale'].notna()]
df_reale.head(2)

,Date,tasso_affollamento_reale
69,2024-10-05,132.1883
70,2024-10-06,132.2567


In [41]:
df_reale.to_csv('../outputs/viz/tasso_reale.csv', index=False)